# Set-up

In [ ]:
# import packages
import random
from time import time

from mace.util import (
    interpolate_by,
    drop_columns,
    sort_by
)

# manipulation 
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# data prep
import missingno as msno
from sklearn.model_selection import train_test_split

# feature engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# modelling
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

from xgboost import XGBRegressor

# metrics
from sklearn.metrics import r2_score, mean_squared_error

# model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# persistence
import joblib
#import dill

RSEED = 394

In [ ]:
# load data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.head()

# 1. Data Inspection

## Size of data

In [ ]:
print(f"Shape of modelling dataset: {df_train.shape}")
print(f"Shape of prediction] dataset: {df_test.shape}")

### Which columns differ? 

In [ ]:
diff_cols = [col for col in df_train.columns if col not in df_test.columns]
diff_cols

## Data features 

In [ ]:
poll_cols = [col for col in df_train.columns if col.find('L3') != -1]
target_cols = [col for col in df_train.columns if col.find('target') != -1]
loc_cols = [col for col in df_train.columns if col.find('Place') != -1 or col.find('Date') != -1 ]
weather_cols = [col for col in df_train.columns if col not in poll_cols + target_cols + loc_cols ]

print(f"Number of columns related to:")
print(f"target: {len(target_cols)}")
print(f"pollution: {len(poll_cols)}")
print(f"location: {len(loc_cols)}")
print(f"weather: {len(weather_cols)}")


### How many places and dates do we have? 

In [ ]:
df_train.Date.unique()
print(f"There are {df_train.Date.nunique()} dates between {df_train.Date.min()} and {df_train.Date.max()}.")

df_train.Place_ID.unique()
print(f"The number of places is {df_train.Place_ID.nunique()}.")

In [ ]:
print(f"We have {df_train.shape[0]} actual observations.")
print(f"If each place had the full 3 months of observations, we should have a sample size of {94 * 340}.")

print(f"\nThis means we do NOT have observations for each place-date pair.")

In [ ]:
place_counts = df_train.groupby('Place_ID').Place_ID.count().unique()
print(f"A place appears between {place_counts.min()} and {place_counts.max()} times.")

### How many pollutants do we have? 

In [ ]:
# list_of_pollutants = [NO2, CH4, ....]
pollutants = list(set([pol.split('_')[1] for pol in df_train[poll_cols].columns]))
pollutants = [pol + "_AI" if pol == "AER" else pol for pol in pollutants]

pollutants

## Target variables

In [ ]:
df_train[target_cols].head()

# 2. Data preparation

## Split train and test

Later in this notebook, we perform within-place imputation of various pollutant-related variables. This means we must split dataset into train and test data such that each city appears _either_ in the train data _or_ the test data (but not both). 

We do this by randomly splitting the list of unique places. Based on this split, we then subset the entire dataset (place-date pairs). 

In [ ]:
df = df_train.copy()

# split dataset into features and target
y = df[['target', 'Place_ID']]          # keep place_ID because we need it for the train/test split
X = df.drop(target_cols, axis = 1)

# obtain unique list of places
places = X.Place_ID.unique()

# split the cities into train and test
places_train, places_test = train_test_split(places, test_size=0.2, random_state=RSEED)

# filter the features for train and test cities
X_train = X[X['Place_ID'].isin(places_train)]
X_test = X[X['Place_ID'].isin(places_test)]

# filter the targets for train and test (+ drop the location variable)
y_train = y[y['Place_ID'].isin(places_train)].drop('Place_ID', axis = 1)
y_test = y[y['Place_ID'].isin(places_test)].drop('Place_ID', axis = 1)

# append features and targets for EDA
train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)

# display the shapes of train and test sets
print(f"Number of places in test is {len(places_test)} and in train {len(places_train)}.")
print("Train set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
print("Split percentage on set:", round(X_train.shape[0]/len(X)*100, 4), "%")


In [ ]:
# drop all columns with satellite position information --> not relevant for prediction
useless_cols = [col for col in poll_cols if col.find('sensor') != -1 or col.find('solar') != -1]
print("The pollution features of interest are:")
print(useless_cols)

poll_cols = [col for col in poll_cols if col not in useless_cols]

X_train = X_train.drop(useless_cols, axis = 1)
X_test = X_test.drop(useless_cols, axis = 1)

# Exploratory data analysis & feature engineering

## Distributions of features

In [ ]:
y_train.describe()

[here should be more code for distributions]

## Relationships among features

tbw

## Missing values

There are no missing values for location and weather variables. 

Polution related features have missing values, which are missing in groups on the pollutant level. For instance, if we have 6 variables for methane, values for an observation are either all missing or all non-missing.

We assess the missing values, finding that missing values are distributed across the entire 3-month period and across the different cities (i.e. we do _not_ have a situation in which only some cities or some months have observations for a pollutant). 

Hence, we decide to use linear interpolation for each pollutant on the city level over time. 

### Inspection

In [ ]:
X_train.isnull().sum()

In [ ]:
# assess missing values
msno.matrix(X_train)

In [ ]:
msno.bar(X_train)

## Dendrogram
From the dendrogram, we see that the majority of pollution data is either fully or highly coherent.
- location and weather data is coherent
- CLOUD data is coherent with the exception of L3_CLOUD_cloud_fraction
- NO2 data has highly coherent but not fully

In [ ]:
# The dendrogram plot provides a tree-like graph generated through hierarchical clustering and groups together columns 
# that have strong correlations in nullity.
msno.dendrogram(X_train)

In [ ]:
# Nota bene: Robert Norris - L3_CO_cloud_height was dropped as it has a different dimension to the other features...
dendrogram_cluster=['L3_O3_cloud_fraction', 
                    'L3_CLOUD_cloud_fraction',
                    'L3_AER_AI_absorbing_aerosol_index',
                    'L3_CLOUD_surface_albedo',
                    'L3_NO2_stratospheric_NO2_column_number_density',
                    'L3_NO2_cloud_fraction',
                    #'L3_CO_cloud_height',
                    'L3_SO2_cloud_fraction',
                    'L3_SO2_absorbing_aerosol_index',
                    'L3_HCHO_tropospheric_HCHO_column_number_density_amf',
                    'L3_NO2_tropospheric_NO2_column_number_density',
                    'L3_CH4_aerosol_optical_depth'
                    ]


In [ ]:
X_train[dendrogram_cluster].describe()

In [ ]:
groups = X_train.sort_values(['Place_ID X Date']).groupby('Place_ID')

focus_place = random.choice(places_train)
highlighted_cluster = random.choice(dendrogram_cluster)
focus_df = groups.get_group(focus_place)

In [ ]:
# Nota bene: Robert Norris - .gca is 'Get the current Axes' which combines all .plot onto same chart...

def plotFeatureHighlightingSinglePlace(
        groups: pd.core.groupby.generic.DataFrameGroupBy, 
        highlighted_place: str, 
        focus_feature: str):
    # Plot for a single feature, highlighting a single place
    for group in groups:
        place=group[0]
        df=group[1][[focus_feature] + ['Date']]
        df.plot(
            kind='line',
            x='Date', xticks=[], xlabel='', 
            figsize=[15,5], ax=plt.gca(),
            color='grey', alpha=0.01,
            legend=False)
        if place == highlighted_place:
            df.plot(
                kind='line',
                x='Date', xticks=[], xlabel='', 
                figsize=[15,5], ax=plt.gca(),
                color='blue',
                legend=False)
        
    plt.show()


In [ ]:
# Plot a single place, highlighting a single feature
def plotPlaceHighlightingSingleFeature(
    focus_df: pd.DataFrame,
    features: list[str], 
    highlighted_feature: str):
    focus_df[dendrogram_cluster + ['Date']].plot(
        kind='line',
        x='Date', xticks=[], xlabel='', 
        figsize=[15,5], ax=plt.gca(),
        color='grey', alpha=0.2,
        legend=False)
    # It is easier to plot all in grey and then choose a single one to highlight than mess with the 'color cycler'...
    focus_df[[highlighted_cluster] + ['Date']].plot(
        kind='line',
        x='Date', xticks=[], xlabel='', 
        figsize=[15,5], ax=plt.gca(),
        color='blue',
        legend=False)
    plt.show()


In [ ]:
# example for a specific place and feature
sample_feature = 'L3_SO2_absorbing_aerosol_index'
sample_place = places_train[6]
sample_place_df = groups.get_group(sample_place)

plotFeatureHighlightingSinglePlace(groups, sample_place, sample_feature)  
plotPlaceHighlightingSingleFeature(sample_place_df, dendrogram_cluster, sample_feature)          


!!MK finished cleaning code here!!

In [ ]:
counts_missing = X.groupby('Place_ID').count().sort_values('L3_CH4_aerosol_height')

counts_missing.head(5)

In [ ]:
# transformers are applied completely separately in parallel
# any columns not listed for any of the transformers will be dropped when remainder='drop' (default)
# cannot 'simply' pickle lambda functions

lmp = Pipeline([
    ('clean', FunctionTransformer(
        drop_columns, 
        feature_names_out='one-to-one', 
        validate=False, 
        kw_args={'columns': useless_cols})),
    ('sort', FunctionTransformer(
        sort_by, 
        feature_names_out='one-to-one', 
        validate=False, 
        kw_args={'columns': ['Place_ID X Date']})),
    ('linear', FunctionTransformer(
        interpolate_by, 
        validate=False,
        kw_args={'columns': ['Place_ID']})),
    ('drop', ColumnTransformer([('drop', 'drop', loc_cols)], remainder='passthrough')),
    ('mean', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('regressor', LinearRegression()) 
])

y_ = df_train[['target']]
X_ = df_train.drop(target_cols, axis = 1)

lmp.fit(X_, y_)
joblib.dump(lmp, 'models/ols.joblib')
#dill.dump(lmp, open("models/ols.dill", "wb"))

### Imputation for missing values
- we decided to use linear interpolation on the level of a place. 
- This means we need to: 
    - order data by place and date
    - interpolate 

In [ ]:
# linear imputation of missing values
X_train.sort_values(['Place_ID X Date'])
X_test.sort_values(['Place_ID X Date'])

# Stepwise imputation: 
# 1. linear interpolation by place (back and forward)

X_train_impute = X_train.groupby('Place_ID').apply(lambda group: group.interpolate(method='linear', limit_direction = 'both')).reset_index(drop = 1)
X_test_impute = X_test.groupby('Place_ID').apply(lambda group: group.interpolate(method='linear', limit_direction = 'both')).reset_index(drop = 1)

# 2. mean fill if no better option using simple imputer
impute_mean = SimpleImputer()
impute_mean.fit(X_train_impute[poll_cols])

X_train_impute[poll_cols] = impute_mean.transform(X_train_impute[poll_cols])
X_test_impute[poll_cols] = impute_mean.transform(X_test_impute[poll_cols])

# check on the one place for which we needed the simple imputer
# X_test_impute[X_test_impute.Place_ID == '5IUK9TG']


In [ ]:
# Nota bene: Robert Norris - groupby seems to perform at least a partial copy so
# we must regroup after imputation...
X_train_impute_groups=X_train_impute.groupby('Place_ID')
place_df = X_train_impute_groups.get_group(sample_place)

plotFeatureHighlightingSinglePlace(X_train_impute_groups, sample_place, sample_feature)  
plotPlaceHighlightingSingleFeature(place_df, dendrogram_cluster, sample_feature)          

In [ ]:
# check that there are no missing values
assert X_train_impute.isnull().sum().sum() == 0
assert X_test_impute.isnull().sum().sum() == 0

In [ ]:
# replace dataset with imputed dataset
X_train = X_train_impute.copy()
X_test = X_test_impute.copy()

# 4. Modelling

## Pipelines

In [ ]:
#---------------------------------------------------
# feature sets 

all_features = weather_cols + poll_cols 

#---------------------------------------------------
# pipelines

scaler = MinMaxScaler()

# linear model
lm = Pipeline([
    ('scaler', scaler),
    ('regressor', LinearRegression()) 
])

# polynomial model
pm = Pipeline([
    ('scaler', scaler), 
    ('tranformer', PolynomialFeatures()),
    ('regressor', LinearRegression())
])

# knn model
knn = Pipeline([
    ('scaler', scaler), # why this one? we can think about it more, could use standard scaler?
    ('regressor', KNeighborsRegressor())
])

# decision tree
dt = Pipeline([
    ('regressor', DecisionTreeRegressor())
])

# random forest
rf = None

# xgboost

#--------------------------------------------------------
# function definitions

def evaluate_model(model, features, Xtest = X_test, Xtrain = X_train): 
    y_test_pred = model.predict(Xtest[features])
    y_train_pred = model.predict(Xtrain[features])
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test  = r2_score(y_test, y_test_pred)
    rmse_train = mean_squared_error(y_train, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test, y_test_pred, squared = False)

    print(f"R2 score on train: {round(r2_train, 3)}")
    print(f"R2 score on test: {round(r2_test, 3)}")
    print("---" * 10)
    print(f"RMSE on train: {round(rmse_train, 3)}")
    print(f"RMSE on test: {round(rmse_test, 3)}")

    return y_test_pred, y_train_pred, rmse_test

## Baseline model

- hypothesis: there is less PM2.5 in the atmosphere when it is windy
- method: OLS

- model
$$ y = b_0 + b_1 * x_1 + \epsilon $$

- estimated model
$$ \hat{PM}_{2.5} = \hat{b}_0 + \hat{b}_1 * wind $$


In [ ]:
base_features = ['relative_humidity_2m_above_ground']

lm.fit(X_train[base_features], y_train)

y_pred_base, _, rmse_base = evaluate_model(lm, base_features)


In [ ]:
# plot y against X
plt.scatter(X_train[base_features], y_train)
plt.xlabel('weather')
plt.ylabel('pm2.5')

In [ ]:
plt.scatter(y_test, y_pred_base)
plt.xlabel('true PM2.5')
plt.ylabel('predicted PM2.5')
plt.show()

## OLS with all features (scaled)

In [ ]:
lm.fit(X_train[all_features], y_train)

y_pred_lm, _, rmse_lm = evaluate_model(lm, all_features)


## KNN regression (scaled)
- tried initially unscaled version with truly *horrible* results

In [ ]:
knn.fit(X_train[all_features], y_train)

y_pred_knn, _, rmse_knn = evaluate_model(knn, all_features)


## Polynomial model

In [ ]:
# pm.fit(X_train[all_features], y_train)

## Simple decision tree (scaled)

In [ ]:
dt.fit(X_train[all_features], y_train)

y_pred_dt, _, rmse_dt = evaluate_model(dt, all_features)


#### DT Model selection

In [ ]:
# splitters = ['best', 'random']
criterion = ['squared_error']
depths = [2,4,_final]
splits = [2,4,_final]
leaves = [4,_final,16]
param_grid = {#"regressor__splitter": splitters,
              "regressor__criterion" : criterion,
              "regressor__max_depth" : depths,
              "regressor__min_samples_split": splits,
              "regressor__min_samples_leaf": leaves
             }
dt_gs = GridSearchCV(dt, param_grid,
                  cv=5, verbose=1, n_jobs=-1)

start = time()
dt_gs.fit(X_train[all_features], y_train)
end = time()
print(f'fit in {end - start}s')


In [ ]:
print('Best score:', round(dt_gs.best_score_, 3))
print('Best parameters:', dt_gs.best_params_)

y_pred_dt_gs, _, rmse_dt_gs = evaluate_model(dt_gs, all_features)

# Ensemble methods

## XGBoost regression

In [ ]:
gb1 = XGBRegressor(
    booster='gbtree',
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0._final,
    colsample_bytree=0._final,
    objective= 'reg:squarederror',
    random_state=RSEED)

xgb = Pipeline([
    #('scaler', scaler),
    ('regressor', gb1)
])

xgb.get_params()

In [ ]:
depths = range(3,10,2)
weights = range(1,6,2)
param_grid = {"regressor__max_depth": depths,
              "regressor__min_child_weight": weights
             }

xgb_gs = GridSearchCV(xgb, param_grid, 
                  cv=5, verbose=1, n_jobs=-1)

start = time()
xgb_gs.fit(X_train[all_features], y_train)
print(f'fit in {time() - start}s')
y_pred_xgb, _, rmse_xgb = evaluate_model(xgb_gs, all_features)

In [ ]:
xgb_gs.best_params_, xgb_gs.best_score_

# Feature importance

In [ ]:
xgb = XGBRegressor(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'reg:squarederror',
    seed=RSEED)

xgb.fit(X_train[all_features], y_train)
y__, _, rmse__ = evaluate_model(xgb, all_features)

xgb.feature_importances_
feature_imp = pd.DataFrame(
    {'features': all_features,
     'importance': xgb.feature_importances_,
    }
)

feature_imp = feature_imp.sort_values('importance',ascending=False).head(20)
best_features = list(feature_imp.features)

# plot best features
plt.barh(feature_imp.features, feature_imp.importance)

In [ ]:
unimp_cols = [col for col in X_train if col not in best_features and col not in loc_cols]
unimp_cols


# Final model

In [ ]:
# transformers are applied completely separately in parallel
# any columns not listed for any of the transformers will be dropped when remainder='drop' (default)
# cannot 'simply' pickle lambda functions

xgb_opt = XGBRegressor(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=7,
    min_child_weight=1,
    gamma=0.2,
    subsample=0.9,
    colsample_bytree=0.6,
    reg_alpha = 100, 
    objective= 'reg:squarederror',
    seed=RSEED)

gs_final = Pipeline([
    ('clean', FunctionTransformer(
        drop_columns, 
        feature_names_out='one-to-one', 
        validate=False, 
        kw_args={'columns': useless_cols + unimp_cols})),
    ('sort', FunctionTransformer(
        sort_by, 
        feature_names_out='one-to-one', 
        validate=False, 
        kw_args={'columns': ['Place_ID X Date']})),
    ('linear', FunctionTransformer(
        interpolate_by, 
        validate=False,
        kw_args={'columns': ['Place_ID']})),
    ('drop', ColumnTransformer([('drop', 'drop', loc_cols)], remainder='passthrough')),
    ('mean', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('regressor', xgb_opt) 
])

## load data
df_train_est = pd.read_csv('data/train.csv')

X_train = df_train_est[df_train_est['Place_ID'].isin(places_train)]
X_test = df_train_est[df_train_est['Place_ID'].isin(places_test)]

y_train = X_train[['target']]
X_train = X_train.drop(target_cols, axis = 1)

y_test = X_test[['target']]
X_test = X_test.drop(target_cols, axis = 1)

gs_final.fit(X_train, y_train)
joblib.dump(gs_final, 'models/gs_final.joblib')

_, _, _ = evaluate_model(
    gs_final, 
    weather_cols + poll_cols + useless_cols + loc_cols,
    X_test, X_train)
